# S&P500 Trend Model

In [87]:
# Import libraries and dependencies
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import datetime
import os

%matplotlib inline

In [88]:
from dotenv import load_dotenv
load_dotenv()
FRED_KEY = os.getenv("FRED_KEY")

In [89]:
from urllib.request import urlopen
import json

def get_jsonparsed_data(url):
    response = urlopen(url)
    data = response.read().decode("utf-8")
    return json.loads(data)

# Data Cleaning

In [90]:
# Pulling time series:
def pull_time_series(time_series_code, dataframe_name):

    url = (f"https://api.stlouisfed.org/fred/series/observations?series_id={time_series_code}&api_key={FRED_KEY}&file_type=json")
    raw_data = get_jsonparsed_data(url)
    dataframe_name = pd.DataFrame(raw_data['observations'])
    
    # convert to datetime
    dataframe_name['Date'] = pd.to_datetime(dataframe_name['date'])
    dataframe_name.set_index('Date', inplace=True)
    dataframe_name = dataframe_name.drop(columns=["date", "realtime_start", "realtime_end"]).sort_index()
    dataframe_name['value'] = pd.to_numeric(dataframe_name['value'],errors='coerce')
    return dataframe_name

In [91]:
# Setting up universal data cleaning functions for FRED data
def clean_data(df,variable_name):
    df.rename(columns={"value":variable_name},inplace=True)
    return df

def pct_change(df,value, time_period):
    df[f'{value}_YoY'] = df[value].pct_change(time_period) * 100
    df[f'{value}_YoY'] = df[f'{value}_YoY'].replace(-np.inf, np.nan).dropna()
    return df

def set_date_to_month_end(df):
    df.index = df.index.to_period('M').to_timestamp('M')
    return df

def resample_monthly(df, variable_name):
    df_resampled = df[variable_name].resample('M').sum()
    df_resampled = pd.DataFrame(df_resampled)
    return df_resampled

In [92]:
# Pull Raw data for model
US_GDP_data = pull_time_series("A191RL1Q225SBEA", "US_GDP_Growth")

US_GDP_df = pull_time_series("GDPC1", "REAL_GDP")
US_Real_PCE_data = pull_time_series("PCEC96", "US_Real_PCE")
US_Initial_Claims_data = pull_time_series("ICSA", "US_Initial_Claims")
US_Cap_Util = pull_time_series("TCU", "US_Capital_Utilization")

US_Real_Disposable_Income = pull_time_series("DSPIC96", "Real_Disposable_Personal_Income")
US_Personal_Savings_Rate = pull_time_series("PSAVERT", "US_Personal_Savings_Rate_%_of_DPI")
US_M2_stock = pull_time_series("M2", "M2_Money_Stock")

### Macro Indicators Processing Section

In [93]:
# Clean PCE data: feature (1 of 6)
clean_data(US_Real_PCE_data, "Real_PCE")
pct_change(US_Real_PCE_data, "Real_PCE", 12)

# Changing date index to last day of the month
US_Real_PCE_data = set_date_to_month_end(US_Real_PCE_data)

# Drop Na's
US_Real_PCE_data = US_Real_PCE_data.dropna()
US_Real_PCE_data.head()

,Real_PCE,Real_PCE_YoY
Date,,
2003-01-31,9219.1,2.643152
2003-02-28,9186.3,1.821104
2003-03-31,9241.2,2.445514
2003-04-30,9287.2,2.436496
2003-05-31,9308.5,3.063620


In [94]:
# Clean US Initial Claims data: feature (2 of 6)
clean_data(US_Initial_Claims_data, "Initial_Claims")

# Resample Weekly data to Monthly
US_Initial_Claims_data_resampled = resample_monthly(US_Initial_Claims_data, "Initial_Claims")

# Calculate YoY change
pct_change(US_Initial_Claims_data_resampled, "Initial_Claims", 12)
US_Initial_Claims_data_resampled.tail()

,Initial_Claims,Initial_Claims_YoY
Date,,
2020-05-31,12329000,1312.256586
2020-06-30,5996000,442.624434
2020-07-31,5475000,540.350877
2020-08-31,5161000,377.428307
2020-09-30,2629000,208.568075


In [95]:
# Clean capacity utilization data: feature (3 of 6)
clean_data(US_Cap_Util, "Capacity_Utilization")

# Calculate YoY Change
pct_change(US_Cap_Util, "Capacity_Utilization", 12)

# Changing date index to last day of the month
US_Cap_Util = set_date_to_month_end(US_Cap_Util)

US_Cap_Util.tail()

,Capacity_Utilization,Capacity_Utilization_YoY
Date,,
2020-04-30,64.0552,-17.629041
2020-05-31,64.6990,-16.834414
2020-06-30,68.6926,-11.586505
2020-07-31,71.1399,-8.116476
2020-08-31,71.4197,-8.244065


In [96]:
# Clean US_Personal_Savings_Rate data: feature (4 of 6)

clean_data(US_Personal_Savings_Rate, "US_Personal_Savings_Rate_%_of_DPI")

# Calculate YoY Change
pct_change(US_Personal_Savings_Rate, "US_Personal_Savings_Rate_%_of_DPI", 12)

# Changing date index to last day of the month
US_Personal_Savings_Rate_df = set_date_to_month_end(US_Personal_Savings_Rate)

US_Personal_Savings_Rate_df.tail(20)

,US_Personal_Savings_Rate_%_of_DPI,US_Personal_Savings_Rate_%_of_DPI_YoY
Date,,
2018-12-31,9.1,37.878788
2019-01-31,8.5,13.333333
2019-02-28,8.6,10.256410
2019-03-31,8.0,2.564103
2019-04-30,7.5,-2.597403
2019-05-31,7.3,-5.194805
2019-06-30,7.1,-10.126582
2019-07-31,7.0,-11.392405
2019-08-31,7.3,-6.410256


In [97]:
# Clean US_Real_Disposable_Income data: feature (5 of 6)

clean_data(US_Real_Disposable_Income, "US_Real_DPI")

# Calculate YoY Change
pct_change(US_Real_Disposable_Income, "US_Real_DPI", 12)

# Changing date index to last day of the month
US_Real_Disposable_Income_df = set_date_to_month_end(US_Real_Disposable_Income)

US_Real_Disposable_Income_df.tail()

,US_Real_DPI,US_Real_DPI_YoY
Date,,
2020-03-31,14949.3,0.630061
2020-04-30,17254.7,16.450476
2020-05-31,16400.8,10.744382
2020-06-30,16107.2,8.635714
2020-07-31,16091.6,8.431770


In [98]:
# Clean M2 Money stock data: feature (6 of 6)
clean_data(US_M2_stock, "M2")

# Resample Weekly data to Monthly (not using resampling function here as we don't want to sum weekly figures, we just want the last figure)
M2_stock_resampled = US_M2_stock.resample('M').last()

# Calculate YoY change
pct_change(M2_stock_resampled, "M2", 12)

M2_stock_resampled.tail(13)

,M2,M2_YoY
Date,,
2019-09-30,15086.4,5.975077
2019-10-31,15189.1,6.605138
2019-11-30,15282.4,7.193760
2019-12-31,15333.6,6.388027
2020-01-31,15445.3,6.724663
2020-02-29,15434.1,6.653169
2020-03-31,16533.3,13.860213
2020-04-30,17406.5,19.313314
2020-05-31,17938.6,22.058693


In [99]:
#Download stock data
def download_data(tickers, start_date):
    stock_data = yf.download(tickers,start=start_date)
    stock_data = stock_data.drop(columns=["Close", "High", "Low", "Open", "Volume"])
    stock_data["Adj Close"] = stock_data['Adj Close'].astype(float)
    stock_data = stock_data.dropna()
    return stock_data

In [100]:
# Download SPY (SP500) Prices
SP500_df = download_data(["SPY"], "1980-01-01")
SP500_df.head()

[*********************100%***********************]  1 of 1 completed


,Adj Close
Date,
1993-01-29,26.079659
1993-02-01,26.265144
1993-02-02,26.320782
1993-02-03,26.599014
1993-02-04,26.710312


In [101]:
# Clean SPY data and calculate monthly returns
SPY_resampled = SP500_df.resample('M').last()
SPY_resampled["Monthly Return"] = SPY_resampled['Adj Close'].pct_change() * 100
SPY_resampled = SPY_resampled.dropna()
SPY_resampled.tail()

,Adj Close,Monthly Return
Date,,
2020-05-31,301.778687,4.764528
2020-06-30,307.130554,1.773441
2020-07-31,325.218140,5.889217
2020-08-31,347.917297,6.979671
2020-09-30,323.500000,-7.018133


### Concating DataFrames

In [102]:
# Not including Real PCE data for now, as data only goes back to 2003
master_df = pd.concat([SPY_resampled,
                       US_Initial_Claims_data_resampled, 
                       US_Cap_Util,
                       US_Real_Disposable_Income,
                       US_Personal_Savings_Rate, 
                       M2_stock_resampled], axis="columns", join='inner')
master_df.tail()

,Adj Close,Monthly Return,Initial_Claims,Initial_Claims_YoY,Capacity_Utilization,Capacity_Utilization_YoY,US_Real_DPI,US_Real_DPI_YoY,US_Personal_Savings_Rate_%_of_DPI,US_Personal_Savings_Rate_%_of_DPI_YoY,M2,M2_YoY
Date,,,,,,,,,,,,
2020-03-31,255.597580,-12.487124,10667000,879.522498,73.5576,-6.174855,14949.3,0.630061,12.9,61.250000,16533.3,13.860213
2020-04-30,288.054260,12.698352,20161000,2238.863109,64.0552,-17.629041,17254.7,16.450476,33.7,349.333333,17406.5,19.313314
2020-05-31,301.778687,4.764528,12329000,1312.256586,64.6990,-16.834414,16400.8,10.744382,24.6,236.986301,17938.6,22.058693
2020-06-30,307.130554,1.773441,5996000,442.624434,68.6926,-11.586505,16107.2,8.635714,19.2,170.422535,18233.5,23.215142
2020-07-31,325.218140,5.889217,5475000,540.350877,71.1399,-8.116476,16091.6,8.431770,17.8,154.285714,18284.5,22.685123


### Building the Model

In [103]:
# Set x variable list of features
x_var_list = ['Initial_Claims_YoY', 
              'Capacity_Utilization_YoY', 
              'US_Real_DPI_YoY', 
              'US_Personal_Savings_Rate_%_of_DPI_YoY', 
              'M2_YoY']

# Filter by x-variable list
master_df[x_var_list].tail()


,Initial_Claims_YoY,Capacity_Utilization_YoY,US_Real_DPI_YoY,US_Personal_Savings_Rate_%_of_DPI_YoY,M2_YoY
Date,,,,,
2020-03-31,879.522498,-6.174855,0.630061,61.250000,13.860213
2020-04-30,2238.863109,-17.629041,16.450476,349.333333,19.313314
2020-05-31,1312.256586,-16.834414,10.744382,236.986301,22.058693
2020-06-30,442.624434,-11.586505,8.635714,170.422535,23.215142
2020-07-31,540.350877,-8.116476,8.431770,154.285714,22.685123


In [104]:
# Shift DataFrame values by 1 = this will shift data points forward one month so that we are using previous months data to predict current month y variable.
master_df[x_var_list] = master_df[x_var_list].shift(1)
master_df[x_var_list].tail()

,Initial_Claims_YoY,Capacity_Utilization_YoY,US_Real_DPI_YoY,US_Personal_Savings_Rate_%_of_DPI_YoY,M2_YoY
Date,,,,,
2020-03-31,17.444444,-1.953392,2.008194,-3.488372,6.653169
2020-04-30,879.522498,-6.174855,0.630061,61.250000,13.860213
2020-05-31,2238.863109,-17.629041,16.450476,349.333333,19.313314
2020-06-30,1312.256586,-16.834414,10.744382,236.986301,22.058693
2020-07-31,442.624434,-11.586505,8.635714,170.422535,23.215142


In [105]:
# Drop NAs and replace positive/negative infinity values
master_df.dropna(subset=x_var_list, inplace=True)
master_df = master_df.replace([np.inf, -np.inf], np.nan)
master_df.head()

,Adj Close,Monthly Return,Initial_Claims,Initial_Claims_YoY,Capacity_Utilization,Capacity_Utilization_YoY,US_Real_DPI,US_Real_DPI_YoY,US_Personal_Savings_Rate_%_of_DPI,US_Personal_Savings_Rate_%_of_DPI_YoY,M2,M2_YoY
Date,,,,,,,,,,,,
1993-03-31,26.948282,2.239895,1404000,-38.805970,81.3902,2.354933,7235.1,2.452005,8.9,-9.183673,3409.7,0.387950
1993-04-30,26.258728,-2.558806,1419000,-18.277066,81.5609,1.424475,7272.7,2.022082,8.7,-8.247423,3411.1,0.252859
1993-05-31,26.966932,2.697024,1718000,-15.182307,81.1656,1.084953,7262.2,2.246622,8.3,-12.121212,3445.5,0.344178
1993-06-30,27.064190,0.360655,1371000,-17.680882,81.1872,0.484934,7248.8,1.610443,7.8,-16.161616,3438.2,1.427730
1993-07-31,26.932825,-0.485382,1754000,-18.149254,81.3220,0.718289,7267.1,0.989161,7.6,-22.772277,3442.3,1.448762


In [106]:
# Set up new dataframe with just x-features and y variable
new_df_list = ['Monthly Return',
               'Initial_Claims_YoY', 
               'Capacity_Utilization_YoY', 
               'US_Real_DPI_YoY', 
               'US_Personal_Savings_Rate_%_of_DPI_YoY', 
               'M2_YoY']
new_df = master_df[new_df_list]
new_df.tail()

,Monthly Return,Initial_Claims_YoY,Capacity_Utilization_YoY,US_Real_DPI_YoY,US_Personal_Savings_Rate_%_of_DPI_YoY,M2_YoY
Date,,,,,,
2020-03-31,-12.487124,17.444444,-1.953392,2.008194,-3.488372,6.653169
2020-04-30,12.698352,879.522498,-6.174855,0.630061,61.250000,13.860213
2020-05-31,4.764528,2238.863109,-17.629041,16.450476,349.333333,19.313314
2020-06-30,1.773441,1312.256586,-16.834414,10.744382,236.986301,22.058693
2020-07-31,5.889217,442.624434,-11.586505,8.635714,170.422535,23.215142


In [107]:
# Construct the dependent variable where if monthly return is greater than 0, then 1, else, 0.
new_df['Monthly Return'] = np.where(new_df['Monthly Return'] > 0, 1.0, 0.0)
new_df.tail()

C:\Users\savag\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Monthly Return,Initial_Claims_YoY,Capacity_Utilization_YoY,US_Real_DPI_YoY,US_Personal_Savings_Rate_%_of_DPI_YoY,M2_YoY
Date,,,,,,
2020-03-31,0.0,17.444444,-1.953392,2.008194,-3.488372,6.653169
2020-04-30,1.0,879.522498,-6.174855,0.630061,61.250000,13.860213
2020-05-31,1.0,2238.863109,-17.629041,16.450476,349.333333,19.313314
2020-06-30,1.0,1312.256586,-16.834414,10.744382,236.986301,22.058693
2020-07-31,1.0,442.624434,-11.586505,8.635714,170.422535,23.215142


In [108]:
new_df['Monthly Return'].value_counts()

1.0    214
0.0    115
Name: Monthly Return, dtype: int64

In [109]:
print(len(new_df))
329*0.7
new_df.iloc[231]

329


Monthly Return                            1.000000
Initial_Claims_YoY                      -11.597633
Capacity_Utilization_YoY                  1.830157
US_Real_DPI_YoY                           3.763472
US_Personal_Savings_Rate_%_of_DPI_YoY    27.536232
M2_YoY                                    9.524125
Name: 2012-06-30 00:00:00, dtype: float64

In [110]:
# Construct training start and end dates
training_start = new_df.index.min().strftime(format= '%Y-%m-%d')
training_end = '2012-06-30'

# Construct testing start and end dates
testing_start =  '2012-07-31'
testing_end = new_df.index.max().strftime(format= '%Y-%m-%d')

# Print training and testing start/end dates
print(f"Training Start: {training_start}")
print(f"Training End: {training_end}")
print(f"Testing Start: {testing_start}")
print(f"Testing End: {testing_end}")

Training Start: 1993-03-31
Training End: 2012-06-30
Testing Start: 2012-07-31
Testing End: 2020-07-31


In [111]:
# Construct the x train and y train datasets
X_train = new_df[x_var_list][training_start:training_end]
y_train = new_df['Monthly Return'][training_start:training_end]


In [112]:
# Construct the x test and y test datasets
X_test = new_df[x_var_list][testing_start:testing_end]
y_test = new_df['Monthly Return'][testing_start:testing_end]

y_test.tail()

Date
2020-03-31    0.0
2020-04-30    1.0
2020-05-31    1.0
2020-06-30    1.0
2020-07-31    1.0
Freq: M, Name: Monthly Return, dtype: float64

In [113]:
# Scaling data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Random Forest Model

In [114]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [120]:
# Fit a SKLearn linear regression using just the training set (X_train, Y_train):
model = RandomForestClassifier(n_estimators=200, max_depth=2, random_state=0)
model.fit(X_train_scaled, y_train)

# Make a prediction of "y" values from the x test dataset
predictions = model.predict(X_test_scaled)

# Assemble actual y data (Y_test) with predicted y data (from just above) into two columns in a dataframe:
Results = y_test.to_frame()
Results["Predicted Value"] = predictions
Results

,Monthly Return,Predicted Value
Date,,
2012-07-31,1.0,1.0
2012-08-31,1.0,1.0
2012-09-30,1.0,1.0
2012-10-31,0.0,1.0
2012-11-30,1.0,1.0
...,...,...
2020-03-31,0.0,1.0
2020-04-30,1.0,0.0
2020-05-31,1.0,0.0


In [121]:
# Function to print a confusion matrix
def cm_to_df(cm, model_title):
    print(f"Confusion Matrix - {model_title}")
    cm_df = pd.DataFrame(cm, index=["Negative", "Positive"], columns=["Predicted Negative", "Predicted Positive"])
    return cm_df

In [122]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, predictions)
cm_to_df(cm, "Random Forest Classifier Confusion Matrix")

Confusion Matrix - Random Forest Classifier Confusion Matrix


,Predicted Negative,Predicted Positive
Negative,0,25
Positive,7,65


In [123]:
print("Test set accuracy: {:.2f}".format(model.score(X_test_scaled, y_test)))

Test set accuracy: 0.67


In [124]:
print("Test set predictions:", model.predict(X_test_scaled))
predictions = model.predict(X_test_scaled)

Test set predictions: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1.
 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0.
 0.]
